In [1]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import process, fuzz
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [2]:
df = pd.read_csv("Downloads/ground_vehicles.csv")

Let's fix the column names

In [3]:
df.columns = df.columns.str.replace(' ', '_')
df["Contract_Number"] = df["Contract_Number_(IDVPIID)"]
df["Award_Number"] = df["Award_Number_(PIID)"]

There seems to be some extraneous data- medical, VA, etc. Let's clean some of that out. 

Let's start with removing any rows that don't pertain to DoD. 

In [4]:
df["contract_ag"] = df['Contracting_Agency'].str.replace(r'\W+', '', regex=True)

In [5]:
df['DoD'] = (df['contract_ag'].str.contains('USDepartmentofDefenseDoD', na=False))

df = df[df.DoD == True]


This data also contains information from FY21, but we are only interested in FY16-FY21

In [6]:
df = df[df.Fiscal_Year != 2021]

Now, lets pull out and label anything that mentions Bradley, Abrams, or Stryker in the Title. 

Who is working on GCS? 

Lets search for specific terms and tag each of the three vehicles:

In [7]:
df['Title'] = df['Title'].str.lower()

df['stryker'] =  np.where(df.Title.str.contains("stryker"), 1,
                   np.where(df.Title.str.contains("armored personnel carrier"),1,  np.NaN))
                        
df['abrams'] =   np.where(df.Title.str.contains("abrams"),1,      
                   np.where(df.Title.str.contains("tank"),1, np.NaN))    
                         
df['bradley']  = np.where(df.Title.str.contains("bradley"), 1,
                   np.where(df.Title.str.contains("fighting vehicle"), 1,  np.NaN))     
                             
df = df.dropna(subset=['stryker','abrams','bradley'], how='all')

df['stryker'] = df['stryker'].fillna(0)
df['abrams'] = df['abrams'].fillna(0)
df['bradley'] = df['bradley'].fillna(0)

len(df.Vendor__Name.unique())


273

There also seems to be a lot of medical data here- we need to remove it manually bc there is a Stryker vendor that makes medical equipment. We can use the PSC_Name codes to filter things out. 

If it leads with an alphabetic character, it signifies a specific category; similarly, if it is a 4-digit code, the first two digits represent a group. 

Finally, I want to scrub any reference to the word "Medical" for anything that has a subcategory in medical. 

In [8]:
df['psc'] = df["PSC_Name"].str.extract(r'(\w+)', expand=False)
df['abb'] = df["psc"].str.extract(r'(\D+)', expand=False)
df['code'] = df["psc"].str.extract(r'(\d{4})', expand=False)
df['pcode'] = df['code'].str.extract(r'(\d{2})', expand=False)

print(df[['PSC_Name','psc', 'abb', 'code', 'pcode']])

                                                PSC_Name   psc  abb  code  \
315                          (53) HARDWARE AND ABRASIVES    53  NaN   NaN   
750    (R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECH...  R425    R   NaN   
755    (J099) MAINT/REPAIR/REBUILD OF EQUIPMENT- MISC...  J099    J   NaN   
756    (J065) MAINT/REPAIR/REBUILD OF EQUIPMENT- MEDI...  J065    J   NaN   
760    (6515) MEDICAL AND SURGICAL INSTRUMENTS, EQUIP...  6515  NaN  6515   
...                                                  ...   ...  ...   ...   
53089           (6640) LABORATORY EQUIPMENT AND SUPPLIES  6640  NaN  6640   
53090  (6515) MEDICAL AND SURGICAL INSTRUMENTS, EQUIP...  6515  NaN  6515   
53091           (6640) LABORATORY EQUIPMENT AND SUPPLIES  6640  NaN  6640   
53092           (6640) LABORATORY EQUIPMENT AND SUPPLIES  6640  NaN  6640   
53093           (6640) LABORATORY EQUIPMENT AND SUPPLIES  6640  NaN  6640   

      pcode  
315     NaN  
750     NaN  
755     NaN  
756     NaN  
760  

In [9]:
df = df[df.abb != 'AN'] #Health R&D
df = df[df.abb != 'Q'] #Medical
df = df[df.pcode != 65] #Medical
df = df[df.psc != 65] #Medical

df['med'] = (df['PSC_Name'].str.contains('MEDICAL', na=False))
df = df[df.med == False]

df['hos'] = (df['PSC_Name'].str.contains('HOSPITAL', na=False))
df = df[df.hos == False]

df['lab'] = (df['PSC_Name'].str.contains('LABORATORY', na=False))
df = df[df.lab == False]

df['endo'] = (df['Title'].str.contains('endoscopy', na=False))
df = df[df.endo == False]

df['op'] = (df['PSC_Name'].str.contains('OPHTHALMIC', na=False))                #troubleshoot later, this should have been taken out ? 
df = df[df.op == False]


Run a quick test to make sure the titles make sense

In [10]:
df.Title.sample(10)

34099    contract w56hzv-17-c-0067 is a sole source con...
48295    spare parts needed in support of the digital e...
10041    431 tension devices, left side for m1 abrams w...
39789    abrams expedited active protection systems (ex...
33777    work directive r-11-13-02 award for retrofit a...
43438    mod to ses/c4isr/cm task order under the syste...
41138                      stryker contractor test support
38721    transmission (hydro-mechanically propelled tra...
41958    systems technical support and sustainment syst...
33500          stryker logistics engineering support (les)
Name: Title, dtype: object

Normalizing Vendors --  We will start by removing spaces, special characters, and strings like LLC or LTD from the end. 

In [11]:
df["ven_name"] = df['Vendor__Name'].str.replace(r'\W+', '', regex=True)
df['ven_name'] = df['ven_name'].apply(str.lower)

df["ven_name"] = df['ven_name'].str.replace('(llc|inc|co|companylimited|ltda|corporation|ltd|corp|company|incorporated|limitedliabilitycompany|limitedliability)$', '', regex=True)


From here, we will do fuzzy matching to group vendors. 

In [12]:
unique_ven = df.ven_name.unique()

score_sort = [(x,) + i
             for x in unique_ven
             for i in process.extract(x, unique_ven, scorer=fuzz.partial_ratio)]

#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['ven_sort','match_sort','score_sort'])

Let's collect only those with high sorting score. (after analysis, only those at 100% were correct)

In [13]:
similarity_sort['sorted_ven_sort'] = np.minimum(similarity_sort['ven_sort'], similarity_sort['match_sort'])

high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 100) &
                (similarity_sort['ven_sort'] !=  similarity_sort['match_sort']) &
                (similarity_sort['sorted_ven_sort'] != similarity_sort['match_sort'])]
high_score_sort = high_score_sort.drop('sorted_ven_sort',axis=1).copy()

high_score_sort.groupby(['ven_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)


,,match_sort
ven_sort,score_sort,
branchsconsulting,100,nch
classicsitesolutions,100,sit
cummins,100,cumminsmidsouth
gan,100,nationalcenterfordefensemanufacturingandmachin...
generaldynamicslandsystem,100,"generaldynamicslandsystems, generaldynamicslan..."
generaldynamicslandsystems,100,generaldynamicslandsystemscustomerservicesuppo...
lockheedmartin,100,"lockheedmartinrotarymissionsystemslmrms, rayth..."
macematerialacquisitioncontractingequipment,100,sit
michigantechnologicaluniversity,100,sit


Let's replace these values in the df

In [14]:
df['ven_name'] = df['ven_name'].replace({'lockheedmartinrotarymissionsystemslmrms':'lockheedmartin',
                                        'generaldynamicslandsystemscustomerservicesupport' : 'generaldynamicslandsystems',
                                        'generaldynamicslandsystemscanada' :  'generaldynamicslandsystems',
                                         'generaldynamicslandsystemss' : 'generaldynamicslandsystems',
                                         'generaldynamicslandsystemssforceprotection' : 'generaldynamicslandsystems',
                                         'lockheedmartin'	: 'raytheonlockheedmartinjavelinjointventure',
                                         'raytheon' : 'raytheonlockheedmartinjavelinjointventure',
                                         'generaldynamicslandsystem': 'generaldynamicslandsystems',
                                         'saabbarracuda' : 'saab',
                                         'optexsystemsholdings': 'optexsystems',
                                         'cumminsmidsouth': 'cummins',
                                        }, regex=True)
ven_list = sorted(df.ven_name.unique())
len(ven_list)

243

264 unique vendors. Lets also clean up the Top Vendor Name list.

In [15]:
df["ven_tname"] = df['Vendor__Top_Name'].str.replace(r'\W+', '', regex=True)
df['ven_tname'] = df['ven_tname'].apply(str.lower)

df["ven_tname"] = df['ven_tname'].str.replace('(llc|inc|co|companylimited|ltda|corporation|ltd|corp|company|incorporated|limitedliabilitycompany|limitedliability)$', '', regex=True)


unique_vent = df.ven_tname.unique()

score_sort = [(x,) + i
             for x in unique_vent
             for i in process.extract(x, unique_vent, scorer=fuzz.partial_ratio)]

#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['vent_sort','match_sort','score_sort'])

similarity_sort['sorted_vent_sort'] = np.minimum(similarity_sort['vent_sort'], similarity_sort['match_sort'])

high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 95) &
                (similarity_sort['vent_sort'] !=  similarity_sort['match_sort']) &
                (similarity_sort['sorted_vent_sort'] != similarity_sort['match_sort'])]
high_score_sort = high_score_sort.drop('sorted_vent_sort',axis=1).copy()

high_score_sort.groupby(['vent_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)



,,match_sort
vent_sort,score_sort,
apiheattransferparent,100,att
att,100,"battlecreekconstruction, plattson"
branchsconsulting,100,nch
classicsitesolutions,100,sit
gan,100,nationalcenterfordefensemanufacturingandmachin...
generaldynamics,100,generaldynamicslandsystem
helixacquisitionholdings,100,sit
lockheedmartin,100,lockheedmartinrotarymissionsystemslmrms
macematerialacquisitioncontractingequipment,100,sit


In [16]:
df['ven_tname'] = df['ven_tname'].replace({'generaldynamics': 'generaldynamicslandsystem',
                                           'lockheedmartinrotarymissionsystemslmrms' : 'lockheedmartin',
                                            'strykerorthopaedics' : 'stryker',
                                           'strykerinternationalholdings' : 'stryker',
                                           'strykercommunications' :'stryker',
                                           'strykergmbhcokg' : 'stryker',
                                           'strykersales' : 'stryker',
                                           'strykersustainabilitysolutions':'stryker',
                                           'strykeruklimited' : 'stryker',
                                           'strykerbv' : 'stryker'
                                        }, regex=True)
ven_list = sorted(df.ven_tname.unique())
len(ven_list)

230

Missing Vendors? 

In [17]:
df[df['ven_name'].isna()] #No Missing Vendors

,ID,Contract_Number_(IDVPIID),Award_Number_(PIID),Modification_Number,Fiscal_Year,Title,Awarded_Amount,Contracting_Office,Contracting_Agency,Place_of_Performance_State,...,abb,code,pcode,med,hos,lab,endo,op,ven_name,ven_tname


Let's look at where the money is going! We can do this by examining the PSC_Name which shows us the category the purchase is charged to. Everything will either have a letter (Abb) or # (code); pcode shows the purchase grouping. 

Let's start by cleaning NaNs. 

In [19]:
df[df['PSC_Name'].isna()] 

,ID,Contract_Number_(IDVPIID),Award_Number_(PIID),Modification_Number,Fiscal_Year,Title,Awarded_Amount,Contracting_Office,Contracting_Agency,Place_of_Performance_State,...,abb,code,pcode,med,hos,lab,endo,op,ven_name,ven_tname
26903,461586923,NaN,W56HZV17C0067_P00056,NaN,2018,abrams sts,NaN,ACC Warren Contracting Center (ACC-WRN),US Department of Defense (DoD),NaN,...,NaN,NaN,NaN,False,False,False,False,False,generaldynamicslandsystemss,generaldynamicslandsystem
32370,552163413,NaN,W56HZV17C0067_P00074,NaN,2019,abrams sts,NaN,ACC Warren Contracting Center (ACC-WRN),US Department of Defense (DoD),NaN,...,NaN,NaN,NaN,False,False,False,False,False,generaldynamicslandsystemss,generaldynamicslandsystem
34954,555097230,NaN,W56HZV17C0067_P00077,NaN,2019,abrams sts,NaN,ACC Warren Contracting Center (ACC-WRN),US Department of Defense (DoD),NaN,...,NaN,NaN,NaN,False,False,False,False,False,generaldynamicslandsystemss,generaldynamicslandsystem
35097,558538537,NaN,W56HZV17C0067_P00079,NaN,2019,abrams sts,NaN,ACC Warren Contracting Center (ACC-WRN),US Department of Defense (DoD),NaN,...,NaN,NaN,NaN,False,False,False,False,False,generaldynamicslandsystemss,generaldynamicslandsystem
35342,561625423,NaN,SPE7L519V2040,NaN,2019,"plug, m1a1 tank barr",NaN,Defense Logistics Agency (DLA),US Department of Defense (DoD),NaN,...,NaN,NaN,NaN,False,False,False,False,False,transhield,transhield
38506,580334521,NaN,W56HZV17C0067_P00088,NaN,2019,abrams sts,NaN,ACC Warren Contracting Center (ACC-WRN),US Department of Defense (DoD),NaN,...,NaN,NaN,NaN,False,False,False,False,False,generaldynamicslandsystemss,generaldynamicslandsystem


Abrams STS -- Found a posting for this actually as R425. https://govtribe.com/opportunity/federal-contract-opportunity/abrams-system-technical-support-sts-and-sustainment-system-technical-support-ssts-follow-on-w56hzv20r0216

for the plug -- lets find the code for plug in what we have (surely there are other plugs!) 

In [20]:
df.at[35342,'PSC_Name'] = '59'
df.at[26903,'PSC_Name'] = 'R'
df.at[32370,'PSC_Name'] = 'R'
df.at[34954,'PSC_Name'] = 'R'
df.at[35097,'PSC_Name'] = 'R'
df.at[38506,'PSC_Name'] = 'R'

In [21]:
plug_test = df[df['Title'].str.contains('abrams sts')]

plug_test[['Title', 'PSC_Name']].head(30)

,Title,PSC_Name
1039,abrams sts work directives - adding funding,"(2350) COMBAT, ASSAULT, AND TACTICAL VEHICLES,..."
7474,the purpose of mod p86 to contract w56hzv-13-c...,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECH...
10065,the purpose of modification p00099 is to incor...,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECH...
11198,the purpose of this modification is to increas...,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECH...
16465,igf::ot::igf. base award of abrams sts/ssts fy...,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECH...
18705,igf::ot::igf this modification adds 2 works di...,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECH...
19527,igf::ot::igf this modification adds 4 work dir...,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECH...
20995,igf::ot::igf this is a sole source contract to...,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECH...
22441,administrative change to paying office. abrams...,(L025) TECHNICAL REPRESENTATIVE- VEHICULAR EQU...
23133,abrams sts dsests ulo to de-obligate excess funds,"(5998) ELECTRICAL AND ELECTRONIC ASSEMBLIES, B..."


Let's smanipulate the PSC_Name. The first character identifies categories of spending, and if there are four digits, the first two specify groups of spending.

In [22]:
def extract_code(row):
    code = str(row['psc'])
    if (code[0].isalpha()):
        # TODO: fix to stop when encountering the first number
        # return ''.join([c for c in code if c.isalpha()])
        return code[:1]
    else:
        return code[:2]

In [23]:
df['psc'] = df["PSC_Name"].str.extract(r'(\w+)', expand=False)

In [24]:
df['code'] = df.apply(extract_code, axis=1)

In [25]:
df[['psc', 'code']].head(10)

,psc,code
315,53,53
750,R425,R
755,J099,J
797,2520,25
807,1290,12
867,6150,61
930,2355,23
946,4220,42
970,4810,48
971,H910,H


Before we dive into comparing, lets consolidate stryker, abrams, and bradley into one column, GCS. 

First lets check the max to see if any rows mentioned multiple. 

In [26]:
df['GCSsum']= df['stryker']+df['bradley']+df['abrams']
df.GCSsum.describe()

count    1647.000000
mean        1.126897
std         0.428679
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         3.000000
Name: GCSsum, dtype: float64

Some rows have both, lets just name that "general". 

In [27]:
df['GCS'] = 'test'
df.loc[df['abrams'] == 1, 'GCS'] = 'abrams'
df.loc[df['stryker'] == 1, 'GCS'] = 'stryker'
df.loc[df['bradley'] == 1, 'GCS'] = 'bradley'
df.loc[df['GCSsum'] >= 2, 'GCS'] = 'general'

df.GCS.describe()

count       1647
unique         4
top       abrams
freq         742
Name: GCS, dtype: object

Let's remove NAs from Awarded Amount so that we can do some overview stats. 

In [28]:
df['Awarded_Amount'] = df['Awarded_Amount'].fillna(0)

df['Awarded_Amount'].describe()

count    1.647000e+03
mean     4.792985e+06
std      3.245477e+07
min     -1.189190e+07
25%      0.000000e+00
50%      8.182132e+04
75%      9.419590e+05
max      7.144557e+08
Name: Awarded_Amount, dtype: float64

Several entries have $0 as the cost- is this accurate or is this data missing? (most likely missing)

Let's replace the 0's with the medians of other billings in the same category (grouping by initial PSC groups). 

In [29]:
df['medpsc'] = df.groupby('code')['Awarded_Amount'].transform('median')

def change_awrd(row):
    awrd = row['Awarded_Amount']
    if awrd == 0:
        return row['medpsc']
    else:
        return awrd

In [30]:
df['Awarded_Amount'] = df.apply(change_awrd, axis=1)

df['Awarded_Amount'].describe()

count    1.647000e+03
mean     4.824561e+06
std      3.245081e+07
min     -1.189190e+07
25%      1.379926e+04
50%      1.412120e+05
75%      9.882935e+05
max      7.144557e+08
Name: Awarded_Amount, dtype: float64

In [31]:
df['award_gain']= df.loc[df['Awarded_Amount'] >= 0,['Awarded_Amount']]
df['award_loss']= df.loc[df['Awarded_Amount'] <= 0,['Awarded_Amount']]
df['award_gain'] = df['award_gain'].fillna(0)
df['award_loss'] = df['award_loss'].fillna(0)


print(df[['award_gain', 'award_loss']])


       award_gain  award_loss
315      15455.00         0.0
750     141212.00         0.0
755      21060.00         0.0
797    3959584.86         0.0
807    1411695.25         0.0
...           ...         ...
49727    13998.48         0.0
49790        0.00   -526794.0
49793   258320.21         0.0
49794   141212.00         0.0
50255   205438.51         0.0

[1647 rows x 2 columns]


Lets run a few SQL queries to get an idea of trends. 

In [32]:
q1 = "SELECT GCS, sum(Awarded_Amount) as aa FROM df GROUP BY GCS ORDER BY aa desc;"
print(pysqldf(q1))

       GCS            aa
0   abrams  5.208164e+09
1  stryker  1.831839e+09
2  bradley  6.113595e+08
3  general  2.946899e+08


A preliminary view shows 

Abrams: 5.2b
Stryker: 1.8b
Bradley: .6b
General: .2b

In [33]:
df.Awarded_Amount.sum()

7946051988.264999

Let's look at the changes from year to year.

In [34]:
q2 = "SELECT GCS,Fiscal_Year, sum(Awarded_Amount) as aa FROM df GROUP BY GCS, Fiscal_Year ORDER BY aa desc;"
print(pysqldf(q2))

        GCS  Fiscal_Year            aa
0    abrams         2018  1.843133e+09
1    abrams         2019  1.641344e+09
2    abrams         2020  1.093988e+09
3   stryker         2019  5.198837e+08
4   stryker         2018  4.553002e+08
5    abrams         2017  4.423467e+08
6   stryker         2016  3.716759e+08
7   stryker         2017  3.358803e+08
8   bradley         2018  2.646469e+08
9    abrams         2016  1.873516e+08
10  bradley         2019  1.722531e+08
11  general         2020  1.495852e+08
12  stryker         2020  1.490990e+08
13  general         2019  1.098635e+08
14  bradley         2016  8.717802e+07
15  bradley         2020  7.079566e+07
16  general         2018  2.860599e+07
17  bradley         2017  1.648584e+07
18  general         2017  3.378858e+06
19  general         2016  3.256438e+06


Notes: 

Abrams dramatically increased between FY17 and FY18 (43m -> 183m);
Stryker consistently uses between 15m and 51m
Bradley's range is between 2m and 26m. 

General GCS funds are between 324,000 and 15m. 2019 and 2020 saw the highest growth for this category. 

Let's now look at the money received and rescinded each year. 

In [35]:
q3 = "SELECT GCS,Fiscal_Year, sum(award_gain) as gain, sum(award_loss) as loss FROM df GROUP BY GCS, Fiscal_Year ORDER BY GCS desc;"
print(pysqldf(q3))

        GCS  Fiscal_Year          gain         loss
0   stryker         2016  3.726073e+08   -931375.71
1   stryker         2017  3.500127e+08 -14132394.72
2   stryker         2018  4.604317e+08  -5131512.15
3   stryker         2019  5.322267e+08 -12342983.52
4   stryker         2020  1.503501e+08  -1251079.46
5   general         2016  3.271318e+06    -14879.76
6   general         2017  3.539501e+06   -160643.19
7   general         2018  3.064302e+07  -2037023.06
8   general         2019  1.134480e+08  -3584478.02
9   general         2020  1.597211e+08 -10135979.75
10  bradley         2016  8.858478e+07  -1406751.63
11  bradley         2017  1.883246e+07  -2346620.41
12  bradley         2018  2.662070e+08  -1560063.60
13  bradley         2019  1.751764e+08  -2923307.89
14  bradley         2020  7.613715e+07  -5341494.98
15   abrams         2016  1.877910e+08   -439401.76
16   abrams         2017  4.437921e+08  -1445455.66
17   abrams         2018  1.848668e+09  -5534961.62
18   abrams 

Many gains and losses each year. (will look better as a graph!)

In [36]:
q4 = "SELECT Contract_Number, ven_name, GCS, sum(award_gain) as gain, sum(award_loss) as loss FROM df GROUP BY Contract_Number, ven_name, GCS ORDER BY gain desc;"
pysqldf(q4).head(10)

,Contract_Number,ven_name,GCS,gain,loss
0,W56HZV18D0012,generaldynamicslandsystemss,abrams,2.463614e+09,-427884.08
1,None,generaldynamicslandsystemss,abrams,1.243312e+09,-31857765.83
2,W56HZV17DB020,generaldynamicslandsystemss,stryker,1.029713e+09,-960146.80
3,None,baesystemslandarmamentslp,bradley,3.849932e+08,-8158618.51
4,None,systemsdrssustainment,abrams,3.066315e+08,0.00
5,None,baesystemslandarmamentslp,general,2.216791e+08,-8120707.00
6,W56HZV16D0060,generaldynamicslandsystemss,stryker,1.810125e+08,0.00
7,W909MY19D0015,raytheonlockheedmartinjavelinjointventure,abrams,1.612287e+08,0.00
8,W56HZV16D0025,generaldynamicslandsystemss,stryker,1.578012e+08,-1222753.00
9,None,generaldynamicslandsystemss,stryker,1.364382e+08,-27519.00


In [37]:
q5 = "SELECT ven_name, GCS, sum(award_gain) as gain, sum(award_loss) as loss FROM df GROUP BY ven_name, GCS ORDER BY gain desc;"
pysqldf(q5).head(10)

,ven_name,GCS,gain,loss
0,generaldynamicslandsystemss,abrams,3.952412e+09,-32688430.45
1,generaldynamicslandsystemss,stryker,1.760390e+09,-17631930.53
2,baesystemslandarmamentslp,bradley,3.850301e+08,-8158618.51
3,systemsdrssustainment,abrams,3.066315e+08,0.00
4,baesystemslandarmamentslp,general,2.216791e+08,-8120707.00
5,allisontransmission,abrams,2.159835e+08,-998026.04
6,raytheonlockheedmartinjavelinjointventure,abrams,1.901607e+08,-9045.49
7,l3technologies,bradley,1.282998e+08,-946350.00
8,honeywellinternational,abrams,1.274804e+08,0.00
9,drsnetworkimagingsystems,abrams,1.041548e+08,-52438.80


In [38]:
q6 = "SELECT Award_Number, ven_name, GCS, sum(award_gain) as gain, sum(award_loss) as loss FROM df GROUP BY Award_Number, ven_name, GCS ORDER BY Award_Number desc;"
pysqldf(q6).head(10)

,Award_Number,ven_name,GCS,gain,loss
0,W91ZRS19C5003,classicsitesolutions,bradley,782482.00,0.00
1,W91ZRS-19-C-5000,larosabuildinggroup,bradley,5490.00,0.00
2,W91QV116C0107,corenicconstructiongroup,abrams,1263320.38,-1043622.43
3,W91CRB19C5007,northropgrummansystems,abrams,7666502.80,0.00
4,W91CRB17C5008,drsnetworkimagingsystems,abrams,6379092.66,0.00
5,W9136418F0160,westernmetalsupplycompany,general,1010777.77,0.00
6,W9133L17F0021,miraclesystems,stryker,1832078.71,0.00
7,W912QR19F0318,themasonhangergroup,abrams,457254.36,0.00
8,W912QR19F0171,themasonhangergroup,abrams,23924.95,0.00
9,W912PB20F3632,centraltexascollege,stryker,3848.86,0.00


In [39]:
pscdf = pd.read_csv("Downloads/psc.csv")

pscdf.columns = pscdf.columns.str.replace(' ', '_')

In [40]:
q7 = "SELECT p.PRODUCT_AND_SERVICE_CODE_NAME as psc, d.GCS, sum(d.award_gain) as ag, sum(d.award_loss) as al FROM df d LEFT JOIN pscdf p ON p.PSC_CODE=d.code GROUP BY psc, GCS ORDER BY ag desc;"
pysqldf(q7).head(5)

,psc,GCS,ag,al
0,"MOTOR VEHICLES, CYCLES, TRAILERS",abrams,3.100645e+09,-19323017.47
1,"MOTOR VEHICLES, CYCLES, TRAILERS",stryker,1.120275e+09,-6786025.13
2,"SUPPORT SVCS (PROF, ADMIN, MGMT)",abrams,5.814455e+08,-15866376.12
3,"MOTOR VEHICLES, CYCLES, TRAILERS",bradley,4.652386e+08,-6114478.83
4,COMM/DETECT/COHERENT RADIATION,abrams,2.949322e+08,0.00


Export for Tableau viz!

In [41]:
df.to_excel('Downloads\df1.xlsx',index=False)

Investigating interesting things!

Intersting thing 1: Fire Equipment?

In [42]:
pd.set_option('display.max_colwidth', None)

In [43]:
fire = df[df['code']=='12']

fire[['Title','PSC_Name','code']].head(10)

,Title,PSC_Name,code
807,"* contract sprdl1-16-c-0015 o awarded 29 aug 13 to real-time laboratories, llc (small business) o 97 each, ccha with 100% option o m1 abrams tank o award amount: $1,564,965.02",(1290) MISCELLANEOUS FIRE CONTROL EQUIPMENT,12
4605,contract for hardware to support the m1a1 abrams tank.,(1240) OPTICAL SIGHTING AND RANGING EQUIPMENT,12
9188,usmc m1a1 tank commander single handle (tcsh),(12) FIRE CONTROL EQUIPMENT,12
10762,"collimator, infinity in support of the m1 abrams tank. (nsn 1240-01-475-0276) procured under farf6.302 urgency",(1240) OPTICAL SIGHTING AND RANGING EQUIPMENT,12
14958,"turret, power control nsn 1240-01-616-2555 in support of the m1 abrams tank. qty: 11; option: 11 (not exercised at time of award). first article test required.",(1240) OPTICAL SIGHTING AND RANGING EQUIPMENT,12
23360,systems technical support for m1a1 abrams. modification to deobligate excess funds after completion of effort.,"(1230) FIRE CONTROL SYSTEMS, COMPLETE",12
24544,"incorporate and exercise option for engineering support for abrams full displacement control grip, travel, prototypes, and protector rws- abrams full displacement control grips.","(1230) FIRE CONTROL SYSTEMS, COMPLETE",12
28006,"switchboard, fire control; nsn 1290-01-358-3928 in support of the m1 abrams tank. qty: 5 option: 5 (not exercised at time of award)",(1290) MISCELLANEOUS FIRE CONTROL EQUIPMENT,12
28386,"switchboard, fire control; nsn 1290-01-600-4747 in support of the m1 abrams tank. qty: 263; option 263 (not exercised at time of award).",(1290) MISCELLANEOUS FIRE CONTROL EQUIPMENT,12
28989,"computer, fire control; nsn 1220-01-619-0117 in support of the m1 abrams tank. qty: 99, option 99 exercised at time of award.",(1220) FIRE CONTROL COMPUTING SIGHTS AND DEVICES,12


In [44]:
fire['psc'].value_counts()

1240    12
1290     8
1220     3
1230     2
12       1
Name: psc, dtype: int64

1240: OPTICAL SIGHTING AND RANGING EQUIPMENT

1290:MISC FIRE CONT EQUIPMENT

In [45]:
fire2 = df[df['code']=='42']

fire2[['Title','PSC_Name','code']].head(10)

,Title,PSC_Name,code
946,4532160749 stryker; auto,(4220) MARINE LIFESAVING AND DIVING EQUIPMENT,42
12182,4535894353 mcs woodland euro stryker system,(4220) MARINE LIFESAVING AND DIVING EQUIPMENT,42
20814,4538734939 radiation detector; stryker mounted,(4220) MARINE LIFESAVING AND DIVING EQUIPMENT,42
21007,4538789931 stryker; auto,(4220) MARINE LIFESAVING AND DIVING EQUIPMENT,42
21030,4538808534 stryker ecp phase 2 - 1.0,(4220) MARINE LIFESAVING AND DIVING EQUIPMENT,42
21316,4538913759 stryker ecp phase 2 - 5.0,(4220) MARINE LIFESAVING AND DIVING EQUIPMENT,42
22213,4539288330 stryker ecp phase 2 - 11`.0,(4220) MARINE LIFESAVING AND DIVING EQUIPMENT,42
22263,4538947161 mini stryker; aut,(4220) MARINE LIFESAVING AND DIVING EQUIPMENT,42
22500,4539399321 radiation detector; stryker mounted - mo,(4220) MARINE LIFESAVING AND DIVING EQUIPMENT,42
24565,4539800966 stryker cover (mcv),(4220) MARINE LIFESAVING AND DIVING EQUIPMENT,42


In [46]:
fire2['psc'].value_counts()

4220    20
4240     2
4210     2
Name: psc, dtype: int64

Okay, we will rename this one as "Marine Lifesaving and Diving Equipment"; upon reading online, it is an attachment/improvement to the Stryker. 

Digging into COMM/DETECT/COHERENT RADIATION

In [47]:
comm= df[df['code']=='58']

comm['psc'].value_counts()

5855    29
5895     5
Name: psc, dtype: int64

We also have some aerospace tags; my hypothesis is that it's a sub category and used in both tanks and airplanes. 

In [48]:
air = df[df['code']=='16']

air[['Title','PSC_Name','code']].head(10)

,Title,PSC_Name,code
28311,"the united states army aviation and missile command has a current effort to provide support to program executive office simulation, training and instrumentation, games for training. the prototype integration facility will build, integrate, deliver and install four stryker virtual collective trainer suites. the system specification requires soldiers being trained to use the virtual battle space 3 and army games for training gaming software packages. the hardware being acquired on 18-207 fulfills this requirement.",(1680) MISCELLANEOUS AIRCRAFT ACCESSORIES AND COMPONENTS,16
29683,mod to change quantity for the joint system integration laboratory customer for the stryker virtual collective training system.,(1680) MISCELLANEOUS AIRCRAFT ACCESSORIES AND COMPONENTS,16
29861,material purchase in support of the stryker virtual collective trainer (svct).,(1680) MISCELLANEOUS AIRCRAFT ACCESSORIES AND COMPONENTS,16
32581,"acquisition of a power control module (pcm) line replaceable unit (lru) interchangeable test adaptor (ita), system control box (scb) lru ita, and a turret power box (tpb) lru ita for a larson test station is necessary to perform manufacturing acceptance of the pcm, scb, and tpb circuitry assemblies on the bradley vehicle.",(1680) MISCELLANEOUS AIRCRAFT ACCESSORIES AND COMPONENTS,16
48998,mod to increase funding for stryker itn support,(1680) MISCELLANEOUS AIRCRAFT ACCESSORIES AND COMPONENTS,16


This is to train people on how to use Stryker. We will keep it (though should it be tagged as Training? )

Digging Deeper: Support Services

In [49]:
sprt = df[df['code']=='R']

sprt[['Title','PSC_Name','code']].head(20)

,Title,PSC_Name,code
750,"the purpose of task order 0003 under contract w56hzv-09-a-a904 is to conduct a business case analysis (bca) evaluating different acquisition strategies relative to the abrams m1e3 modernization program. the bca will include a comparison of costs, timeliness, technical performance, risks, and programmatic issues associated with each acquisition strategy to include any factors that may influence the acquisition decision. modification 05 revises the administrating office from dcma huntsville to dcma orlando.",(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,R
1611,igf::ot::igf modification to exercise option term i for milestone documentation supporting armored multi-purpose vehicle (ampv) and bradley engineering change proposal 2 (ecp 2)..,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,R
1977,bradley system technical support p00045,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,R
2602,bradley system technical support p00046,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,R
3327,igf::ct::igf decrease funding to remove expired funds for stryker program option year six,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,R
3668,bradley system technical support p00047,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,R
3749,igf::ot::igf pd mbts/pm abrams vehicle health management system network integraton evaluation 16.2/17.1,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,R
4013,igf::ot::igf stryker - nbcrv opt yr 1,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,R
4703,igf::ot::igf predictive analytics assessment for abrams tank,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,R
4755,igf::ct::igf to fund period 1 [fiscal year 2016 (fy16)] stryker wholesale supply in accordance with the contract terms established under stryker requirements contract w56hzv-16-d-0060,(R499) SUPPORT- PROFESSIONAL: OTHER,R


Lets address igf::ct::igf

In [50]:
df['igf'] =  np.where(df.Title.str.contains("igf::ct::igf"), 'ct',
                   np.where(df.Title.str.contains("igf::ot::igf"),"ot",  
                     np.where(df.Title.str.contains("igf::ot::igf"),"cl",        
                            np.NaN)))

In [51]:
df['igf'].value_counts()

nan    1439
ot      189
ct       19
Name: igf, dtype: int64

What are the critical functions?

In [52]:
ct = df[df['igf']=='ct']

ct[['Contract_Number','Title','PSC_Name','code','igf','GCS']].head(20)

,Contract_Number,Title,PSC_Name,code,igf,GCS
971,W91CRB14D0031,igf::ct::igf incremental funding for aec's omnibus contract to support the stryker program base year - months 1-12,(H910) OTHER QC/TEST/INSPECT- WEAPONS,H,ct,stryker
2497,W91CRB14D0031,igf::ct::igf incremental funding to support aec's omnibus contract that supports the stryker program. base year - months 1-12,(H910) OTHER QC/TEST/INSPECT- WEAPONS,H,ct,stryker
2499,W91CRB14D0024,igf::ct::igf incremental funding to support aec's omnibus contract for the bradley and mrap programs base - labor (months 1-12),(H910) OTHER QC/TEST/INSPECT- WEAPONS,H,ct,bradley
3327,W91CRB06D0015,igf::ct::igf decrease funding to remove expired funds for stryker program option year six,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,R,ct,stryker
4755,W56HZV16D0060,igf::ct::igf to fund period 1 [fiscal year 2016 (fy16)] stryker wholesale supply in accordance with the contract terms established under stryker requirements contract w56hzv-16-d-0060,(R499) SUPPORT- PROFESSIONAL: OTHER,R,ct,stryker
7296,W91CRB14D0024,igf::ct::igf decrease funding on aec's omnibus contract that supports the stryker program. base - labor (months 1-12),(H910) OTHER QC/TEST/INSPECT- WEAPONS,H,ct,stryker
10943,W91CRB14D0031,igf::ct::igf incremental funding for aec's omnibus contract that supports the stryker program option year 1 - months 13-24,(H910) OTHER QC/TEST/INSPECT- WEAPONS,H,ct,stryker
17075,NaN,igf::ct::igf stryker fuel spill cleanup on i-5,(F108) ENVIRONMENTAL SYSTEMS PROTECTION- ENVIRONMENTAL REMEDIATION,F,ct,stryker
19564,W31P4Q09A0021,igf::ct::igf exercise options; obsolescence support to custom and borders patrol; obsolescence support to bradley family of fighting vehicle; obsolescence support to amc counterfeit practices ipt,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,R,ct,bradley
20040,NaN,igf::ct::igf. increase funding for the abrasive clean of stryker hulls.,"(J023) MAINT/REPAIR/REBUILD OF EQUIPMENT- GROUND EFFECT VEHICLES, MOTOR VEHICLES, TRAILERS, AND CYCLES",J,ct,stryker


In [53]:
ot = df[df['igf']=='ot']

ot[['Contract_Number','Title','PSC_Name','code','igf','GCS']].head(20)

,Contract_Number,Title,PSC_Name,code,igf,GCS
755,NaN,igf::ot::igf lgcc stryker neptune loa mod,(J099) MAINT/REPAIR/REBUILD OF EQUIPMENT- MISCELLANEOUS,J,ot,stryker
1611,W56HZV15AA901,igf::ot::igf modification to exercise option term i for milestone documentation supporting armored multi-purpose vehicle (ampv) and bradley engineering change proposal 2 (ecp 2)..,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,R,ot,bradley
2167,W58RGZ12D0089,igf::ot::igf task order 0538- stryker wireless internal combustion engine (wice)adapter/extension cables,(J016) MAINT/REPAIR/REBUILD OF EQUIPMENT- AIRCRAFT COMPONENTS AND ACCESSORIES,J,ot,stryker
2596,W56HZV13D0015,igf::ot::igf cls/training services for m1a1 tanks and m88 recovery vehicles for the government of iraq for jan-dec 2016,(Z1EE) MAINTENANCE OF TANK AUTOMOTIVE FACILITIES,Z,ot,abrams
2828,NaN,igf::ot::igf preventative maintenance for stryker ambulance cots,(J042) MAINT/REPAIR/REBUILD OF EQUIP- FIRE FIGHTING/RESCUE/SAFETY EQUIP; ENVIRON PROTECT EQUIP/MATLS,J,ot,stryker
3499,NaN,igf::ot::igf; modification to determine the feasibility of integrating a sample probe onto the ngcd 3 (aka. multi-sample identifier) system and possible future integration onto the stryker nbcrv.,(AC93) R&D- DEFENSE SYSTEM: MISCELLANEOUS HARD GOODS (ADVANCED DEVELOPMENT),A,ot,stryker
3749,N0002412D6404,igf::ot::igf pd mbts/pm abrams vehicle health management system network integraton evaluation 16.2/17.1,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,R,ot,abrams
3797,NaN,igf::ot::igf this contract is for up to three years of manufacturing technical assistance services to be performed conus and oconus in support of the m1a1 tank production effort at the egyptian tank plant.,(L099) TECHNICAL REPRESENTATIVE- MISCELLANEOUS,L,ot,abrams
4013,W911SR10D0004,igf::ot::igf stryker - nbcrv opt yr 1,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,R,ot,stryker
4223,NaN,igf::ot::igf modification for addition of integrating a sample probe into the ngcd msid and integration onto the stryker; extending blind test in brassboard by 2 weeks,(AC93) R&D- DEFENSE SYSTEM: MISCELLANEOUS HARD GOODS (ADVANCED DEVELOPMENT),A,ot,stryker


These seem to be funding increases, for the most part. What is the total sum of CT? 

In [54]:
ct.Awarded_Amount.sum() #183m

183011789.42

It's interesting to see two oil spills in this list.Lets do a quick check on environmental remediation efforts  that are tied to GCS.

In [55]:
env = df[df['psc']=='F108']
env[['Fiscal_Year','Title','PSC_Name','GCS']].head(20)

,Fiscal_Year,Title,PSC_Name,GCS
17075,2017,igf::ct::igf stryker fuel spill cleanup on i-5,(F108) ENVIRONMENTAL SYSTEMS PROTECTION- ENVIRONMENTAL REMEDIATION,stryker
20971,2017,igf::ct::igf stryker fuel spill cleanup on i-5,(F108) ENVIRONMENTAL SYSTEMS PROTECTION- ENVIRONMENTAL REMEDIATION,stryker


Okay good! It's only stryker. How much did it cost? 

In [56]:
env.Awarded_Amount.sum() #4k 

4079.33

Let's dig into training aids some more

In [57]:
tr = df[df['code']=='69']
tr[['Fiscal_Year','Title','PSC_Name', 'Awarded_Amount','GCS']].head(10)

,Fiscal_Year,Title,PSC_Name,Awarded_Amount,GCS
2336,2016,"the contract awards sts services to oasis to modify, revise, and update the common software library for the m1 abrams tank and the bradley fighting vehicle.",(6910) TRAINING AIDS,2354569.75,general
2345,2016,stryker mgs mobile agts pop extension,(6920) ARMAMENT TRAINING DEVICES,0.00,stryker
6599,2016,"stryker semitendinosus tendon, tibialis tendon anterior, gracilis tendon",(6910) TRAINING AIDS,39575.00,stryker
10244,2016,"m1a1 mobile platoon agts - extend the pop for slins 0003aa, 0003ab and 0003ac at no cost to allow the contractor to provide a plan on how they intend to fulfill outstanding terms and conditions to ""design, develop, procure, integrate, and test the hardware and software required to supply one (1) tdt that replicates driver operations in accordance with the m1a1 tank driver trainer (tdt) specification as approved in the tim and ipts.",(6910) TRAINING AIDS,0.00,abrams
10643,2017,stryker mobile gun system (mgs) advanced gunnery training system (agts) gunnery maual tc 3-20.31 software upgrade,(69) TRAINING AIDS AND DEVICES,0.00,stryker
11078,2016,pm gctt has a requirement to upgrade the stryker maintenance training system (mts) software by converting the lessons on the diagnostic troubleshooting trainer (dtt) from a 2-d format to a 3-d format. w900kk-09-d-0562 delivery order 0002 was awarded to design and develop maintenance lessons in 3-d format for the stryker mobile gun system (mgs). this requirement will deliver the conversion of the current lessons that are 2-d to a 3-d graphic design.,(6910) TRAINING AIDS,4442412.00,stryker
11278,2016,i-miles cvtess system bradley,(6910) TRAINING AIDS,-1179955.66,bradley
11430,2016,i-miles cvtess system bradley,(6910) TRAINING AIDS,1984313.66,bradley
12892,2017,stryker mobile gun system (mgs) advanced gunnery training system (agts) gunnery maual tc 3-20.31 software upgrade,(69) TRAINING AIDS AND DEVICES,0.00,stryker
13897,2017,igf::ot::igf services-priced. abrams csl upgrade ordering period 09 december 2016 to 08 december 2017.,(6910) TRAINING AIDS,1310400.00,abrams


Main training for each: 

i-miles cvtess = Bradley (later adding Abrams in 2020) |
mgs mobile agts = Stryker |
ed/tt(ft benning) = Abrams

In [58]:
df['train'] =  np.where(df.Title.str.contains("i-miles cvtess"), 'bradley',
                   np.where(df.Title.str.contains("mgs mobile agts"),"stryker",  
                     np.where(df.Title.str.contains("ed/tt"),"abrams",        
                            np.NaN)))

In [59]:
train = df[['train','Title','award_gain']]

train.groupby('train').sum()

C:\Users\kalia\AppData\Local\Temp\ipykernel_44072\3850486589.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  train.groupby('train').sum()


,award_gain
train,
abrams,1.700237e+07
bradley,2.164841e+06
nan,8.030386e+09
stryker,0.000000e+00


Looking at Contracting Offices, Top Vendors

In [60]:
q8 = "SELECT Contracting_Office as co, sum(Awarded_Amount) as aa FROM df GROUP BY co ORDER BY aa desc;"
pysqldf(q8).head(10)

,co,aa
0,0646 AQ TM CONTRACTING TEA - APO,6.017419e+09
1,US Department of the Army,9.846085e+08
2,DLA Land - Warren,3.723811e+08
3,ACC-APG Aberdeen - Division C,2.451065e+08
4,ACC New Jersey Contracting Center (ACC-PICA),9.664224e+07
5,"ACC-APG Orlando, FL",7.658229e+07
6,ACC Contracting Centers,3.782974e+07
7,DLA Land - Aberdeen,3.091885e+07
8,ACC Aberdeen Proving Ground Contracting Center (ACC-APG) (includes former Fort Monmouth),1.968142e+07
9,Defense Microelectronics Activity (DMEA) (formerly Advanced Microelectronics Section under USAF),1.767431e+07


In [61]:
df['co'] = np.where(df.Contracting_Office.str.contains("DLA Land"), 'DLA',
                   np.where(df.Contracting_Office.str.contains("ACC"), 'ACC',
                     np.where(df.Contracting_Office.str.contains("US Department of the Army"), "Army",
                        np.where(df.Contracting_Office.str.contains("0646 AQ TM"), 'APO',
                            np.NaN))))

In [62]:
co= df[['co','Awarded_Amount']]

co.groupby('co').sum()

,Awarded_Amount
co,
ACC,4.870232e+08
APO,6.017419e+09
Army,9.846085e+08
DLA,4.038241e+08
nan,5.317751e+07


In [63]:
q9 = "SELECT co, ven_name, sum(Awarded_Amount) as aa FROM df GROUP BY co, ven_name ORDER BY aa desc;"
pysqldf(q9).head(10)

,co,ven_name,aa
0,APO,generaldynamicslandsystemss,4.653279e+09
1,Army,generaldynamicslandsystemss,8.564627e+08
2,APO,baesystemslandarmamentslp,5.443362e+08
3,APO,systemsdrssustainment,3.141607e+08
4,ACC,raytheonlockheedmartinjavelinjointventure,2.011158e+08
5,DLA,generaldynamicslandsystemss,1.742751e+08
6,APO,allisontransmission,1.673401e+08
7,APO,honeywellinternational,1.270323e+08
8,APO,l3technologies,8.928452e+07
9,ACC,kongsbergdefenceaerospaceas,8.686852e+07


Looking at top vendors for top PSC 

In [64]:
df['psctop'] = np.where(df.code.str.contains("R"), 'support',
                    np.where(df.code.str.contains("25"), 'veh comp',
                              np.where(df.psc.str.contains("4855"), 'night vis',
                                    np.where(df.code.str.contains("J"), 'maint',
                                       np.where(df.code.str.contains("A"), 'RD',
                                               np.where(df.code.str.contains("U"), 'edu', 
                            np.NaN))))))                

In [65]:
q10 = "SELECT psctop, ven_name, sum(Awarded_Amount) as aa FROM df WHERE psctop='night vis' GROUP BY psctop, ven_name  ORDER BY aa desc;"
pysqldf(q10).head(70)

,psctop,ven_name,aa


In [66]:
support = df[df['psc']== 'R425']

support.Title.value_counts()

contract w56hzv-17-c-0067 is a sole source contract to general dynamics land systems (gdls) for the acquisition of system technical support (sts) and sustainment system technical support (ssts) services. sts and ssts services consist of engineering and logistics services to the abrams family of vehicles (fov).                                                                                          68
stryker contractor test support                                                                                                                                                                                                                                                                                                                                                                                  35
systems technical support and sustainment systems technical support services, consisting of engineering, logistics, and fielding support for the bradley fighting vehicle family of vehicles, th

In [67]:
maint = df[df['code']=='J']

maint = maint[['Title', 'Awarded_Amount']]

maint.groupby('Title').sum()

,Awarded_Amount
Title,
abrams,-190758.32
"award contract line item number 0211 for program manager bradley support on task order 0019 (gmdt). gunnery maintenance and device trainers (gmdt) provides training and technical assistance to the gunnery, maintenance, and driver trainers (gmdt) family of devices to include, but not limited to the following devices: bradley advanced training system (bats); conduct of fire trainer (coft) family of devices; army advanced gunnery training system (agts) family of devices; common driver trainer (cdt) family of devices; abrams, bradley, and stryker maintenance training system (mts); construction equipment virtual trainers (cevt); and gunnery and maintenance desktop trainers for on demand device support across the gmdt portfolio.",28513.00
"award contract line items 0075, 0076, 0077 for task order 0019 depot support (gmdt). gunnery maintenance and device trainers (gmdt) provides training and technical assistance to the gunnery, maintenance, and driver trainers (gmdt) family of devices to include, but not limited to the following devices: bradley advanced training system (bats); conduct of fire trainer (coft) family of devices; army advanced gunnery training system (agts) family of devices; common driver trainer (cdt) family of devices; abrams, bradley, and stryker maintenance training system (mts); construction equipment virtual trainers (cevt); and gunnery and maintenance desktop trainers for on demand device support across the gmdt portfolio.",168516.00
base contract support for annual stryker retrofit activity in various locations.,5966047.00
contractor support for government testing of fbh and dvh strykers at various government test sites.,-193223.00
...,...
"three year team armor program (tap)contract to include one base year with two option periods for 12-months each for the receipt, inspection, testing, diagnosis, repair, storage, issue parts in support of abrams, bradley and wolverine vehicles.",14305951.70
"wd r-11-02-01 and r-11-04-01 for stryker retrofits in vilseck germany and ft. carson, mo.",222115.00
"wd r-11-10-10 for stryker retrofits in ft. pickett, va.",23056.00


In [68]:
edu = df[df['code']=='U']

edu = edu[['Awarded_Amount', 'PSC_Name']]

edu.groupby('PSC_Name').sum()

,Awarded_Amount
PSC_Name,
(U008) EDUCATION/TRAINING- TRAINING/CURRICULUM DEVELOPMENT,3.178150e+04
(U009) EDUCATION/TRAINING- GENERAL,1.500212e+04
(U013) EDUCATION/TRAINING- COMBAT,1.372642e+08
(U099) EDUCATION/TRAINING- OTHER,-5.986748e+04


In [69]:
support = df[df['award_loss']< 0 ]

support[['Title', 'code', 'award_loss']].head(10)

,Title,code,award_loss
1672,"the purpose of this modification is to implement the following changes to the contract: a. add 108 ea rear sentry seats, p/n 10695231-110 to the icv-icvv reclaim list (attachment 0083). b. obtain a credit in the amount of $389,448 against the dvh stryker exchange infantry carrier vehicle (xicvvs) clins 0601ca, 0601cb and 0601cc to account for the 108 ea rear sentry seats, p/n 10695231-110 being reclaimed in the exchange process. c. obtain a credit against clin 0601ca for the srat ii packaging issue and the exchange parts for which the contractor was unable to furnish under the exchange firm fixed price in the amount of $418,298. please reference pco letters sjb408, sjb409, sjb410, rkb136, rkb138, rkb147 and the srat ii packaging credit email, dated 14 may 2015.",23,-823353.00
1794,"noun: wiring harness, bran. nsn: 6150-01-465-3770. part number: 12388468-3. quantity: 138 each. vehicle: tank, abrams, m1a2 unique. modification extends the delivery schedule date of the purchase order.",61,-877.68
2720,cable and conduit a for dsests and will be utilized on the m1a1 (4000 characters) abrams and bradley. nsn 6150014786962,61,-6640.00
3327,igf::ct::igf decrease funding to remove expired funds for stryker program option year six,R,-10756.32
4013,igf::ot::igf stryker - nbcrv opt yr 1,R,-57274.06
6246,"engineering design, development, integration, and test of cummins 675hp engine in support of the bradley ecp effort.",28,-223391.49
7287,igf::ot::igf modification for tower 3 temp tanks design and to deduct work at tower 3 repair water towers at nsf indian head,Z,-438524.08
7296,igf::ct::igf decrease funding on aec's omnibus contract that supports the stryker program. base - labor (months 1-12),H,-38197.94
7525,gunner restraint system (delta) for the stryker icv double v,23,-285.13
9673,"4605 - ngats - next generation automatic test set - limited user test 2. the ngats v6 is intended to be the newest off-platform automatic test system (ats) in the army integrated family of test equipment (ifte) product line that will provide the soldier a weapon system diagnostic tool. the ngats v6 is the replacement for the aging direct support system test set (dsests), by providing diagnostic support for current weapons system platforms supported by dsests which supports the abram and bradley platforms, the base shop test facility (bstf)(v)3, and bstf(v)5 which supports aviation and ground missile system. the bstf (v) 3, and bstf (v) 5 capability will each be added incrementally as pre-planned product improvements (p3i). up-armored m1085 medium tactical vehicle (mtv) as the prime mover. ngats was supported under the test support contract (tsc), which is a cost plus award fee indefinite delivery indefinite quantity (id-iq) contract w9115u-07-d-0003.",R,-3404.48


One concern is PoP extension. Lets see how often this happened. 

In [70]:
df['pop'] = np.where(df.Title.str.contains("pop"), 'pop', np.nan)

df['pop'].value_counts()

nan    1634
pop      13
Name: pop, dtype: int64

There are 13 PoP comments. Lets see more about them. 

In [71]:
pop = df[df['pop']=='pop']

pop = pop[['Title', 'Modification_Number', 'Contract_Number', 'Fiscal_Year', 'ven_name', 'Awarded_Amount', 'PSC_Name']]

pop.head(13)

,Title,Modification_Number,Contract_Number,Fiscal_Year,ven_name,Awarded_Amount,PSC_Name
2203,modification to the current stryker test delivery order to extend the pop on clin 0423aj at no additional cost to the government.,11,W56HZV13D0008,2016,generaldynamicslandsystemss,12831.00,"(2355) COMBAT, ASSAULT, AND TACTICAL VEHICLES, WHEELED"
2345,stryker mgs mobile agts pop extension,13,W900KK09D0314,2016,raytheonlockheedmartinjavelinjointventure,0.00,(6920) ARMAMENT TRAINING DEVICES
5092,1. this is a bilateral modification pursuant to mutual agreement. 2. the purpose of modification 65 to contract w56hzv-13-d-0008 is to clarify the period-of-performance (pop) for engineering support in production (esip) ordered under dvh stryker vehicle subclins identified below: (a) 0601ba- quantity of 24 - dvh stryker exchange infantry carrier vehicles (xicvvs); (b) 0601ca- quantity of 45 - dvh stryker exchange infantry carrier vehicles (xicvvs); (c) 0601cb- quantity of 5 - dvh stryker exchange infantry carrier vehicles (xicvvs); (d) 0601cc- quantity of 4 - dvh stryker exchange infantry carrier vehicles (xicvvs); (e) 0602ba- quantity of 28 - dvh stryker exchange mortar carrier vehicles (xmcvvs); (f) 0602ca- quantity of 17 - dvh stryker exchange mortar carrier vehicles (xmcvvs); (g) 0603ba- quantity of 17 - dvh stryker exchange commanders vehicles (xcvvs); (h) 0603ca- quantity of 22 - dvh stryker exchange commanders vehicles (xcvvs); (i) 0604ba- quantity of 5 - dvh stryker fire support vehicles (xfsvvs); (j) 0604ca- quantity of 7 - dvh stryker fire support vehicles (xfsvvs); (k) 0605ca- quantity of 5 - dvh stryker exchange engineer squad vehicles (xesvvs); (l) 0605cb- quantity of 10 - dvh stryker exchange engineer squad vehicles (xesvvs); (m) 0606ba- quantity of 19 - dvh stryker exchange medical evacuation vehicles (xmevvs); (n) 0606ca- quantity of 5 - dvh stryker exchange medical evacuation vehicles (xmevvs); (o) 0606cb- quantity of 6 - dvh stryker exchange medical evacuation vehicles (xmevvs); (p) 0606cd- quantity of 12 - dvh stryker exchange medical evacuation vehicles (xmevvs); (q) 0607ca- quantity of 6 - dvh stryker exchange anti-tank guided missile vehicles (xatvvs); (r) 0607cb- quantity of 8 - dvh stryker exchange anti-tank guided missile vehicles (xatvvs).,65,W56HZV13D0008,2016,generaldynamicslandsystemss,12831.00,"(2355) COMBAT, ASSAULT, AND TACTICAL VEHICLES, WHEELED"
7009,the purpose of this modification is to extend the contract pop to 28 feb 2017 and fund wd 2086 for korea abrams fsr-oconus. igf::ot::igf,P00087,NaN,2016,generaldynamicslandsystemss,149948.00,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL
10244,"m1a1 mobile platoon agts - extend the pop for slins 0003aa, 0003ab and 0003ac at no cost to allow the contractor to provide a plan on how they intend to fulfill outstanding terms and conditions to ""design, develop, procure, integrate, and test the hardware and software required to supply one (1) tdt that replicates driver operations in accordance with the m1a1 tank driver trainer (tdt) specification as approved in the tim and ipts.",P00011,NaN,2016,raytheonlockheedmartinjavelinjointventure,0.00,(6910) TRAINING AIDS
33504,base contract: stryker class requirements contract. task order: 0031: established for fy15 reset services. mod 11 to 0031: issued to extend delivery schedule for 2 mgs vehicles and pop for reset line shutdown.,11,W56HZV13D0008,2018,generaldynamicslandsystemss,12831.00,"(2355) COMBAT, ASSAULT, AND TACTICAL VEHICLES, WHEELED"
35457,"abrams sts. sole source to general dynamics land system. extending pop dates, funding overruns, and making administrative changes.",P00069,NaN,2019,generaldynamicslandsystemss,2118426.00,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL
36393,stryker sts - pm yr 3 pop extension,1,W56HZV16D0037,2019,generaldynamicslandsystemss,141212.00,(R425) SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL
37684,abrams sts funding increase and pop extension.,P00078,NaN,2019,generaldynamicslandsys